# Intelligent Data Development

### Project 1: Predicting Orders for Glovo

### By Andrew Bennett, Edward Monbiot, Alvaro Ortiz & Mariam Sargsyan




Imagine you just joined Glovo. Glovo follows a slot-based system for the couriers to fullfil the orders that come in. For simplification, you can imagine those slots are non-overlapping hours, so that every every city has 24 slots every day, one for each hour. Glovo needs to know the optimal number of couriers that are needed on every hour slot of every city. Too many couriers, and there will be many idle couriers not earning money. Too few couriers, and orders will have to wait to be processed, leading to higher delivery times.

At the moment, Operations decides manually how many couriers are needed, based on past demand. As the number of cities grows, this becomes unsustainable. They want to automate the process by which they decide how many courier-slots should be opened every hour. For simplification, we can assume that every Sunday at midnight, we need to know how many couriers we need for every hour of the week that is starting. That means that if today is Sunday, May 8th 23:59, they want us to know how many orders will be placed every hour of the week that goes from May 9th 00:00 to May 15th 23:00, both included. Every Sunday, you can use all data from that week to forecast the next one.

This problem has many steps, but we will keep this project to the order forecast for one city: we want to know, for one city and every Sunday, how many orders we're going to receive on every hour of the upcoming week.

Load the file data_BCN.csv

Explore the data, visualise it. Look for trends, cycles and seasonalities. Also, can you find any outliers? days or hours that break those patterns?


In [82]:
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px
from sklearn.model_selection import train_test_split
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import holidays
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings 

warnings.filterwarnings('ignore')


# Table of Contents

1. [Import Data](#Import-Data)
2. [Data Inspection](#Data-Inspection)
3. [Train Test Split](#Train-Test-Split)
4. [EDA Train Data Set](#EDA-Train-Data-Set)
    1. [Analysis of Peak Order Times and Dates](#Analysis-of-Peak-Order-Times-and-Dates)
    2. [Shift Analysis](#Shift-Analysis)
    3. [Meal Times Analysis](#Meal-Times-Analysis)
    4. [Zero Order Analysis](#Zero-Order-Analysis)
    5. [Seasonality Analysiss](#Seasonality-Analysis)
9. [Modelling](#Modelling)
    1. [1. Random Forest](#1.-Random-Forest)
    2. [1.2 Random Forest with zero orders](#1.2-Random-Forest-with-zero-orders)
    3. [2. Ensembles](#2.-Ensembles)
    4. [2.1 Boosting](#2.1-Boosting)
    5. [2.2 Stacking](#2.2-Stacking)
    6. [2.3 XgBoost](#2.3-XgBoost)

# Import Data

In this part we will just import the dataset

In [83]:


data = pd.read_csv('./data_BCN.csv')


# Data Inspection

Moreover we will do a inspection of the data to check the format, the columns, the description of the data and simple trends

In [84]:
data.head(10)

,time,orders,city
0,2021-02-01 0:00:00,0.0,BCN
1,2021-02-01 1:00:00,0.0,BCN
2,2021-02-01 2:00:00,0.0,BCN
3,2021-02-01 3:00:00,0.0,BCN
4,2021-02-01 4:00:00,0.0,BCN
5,2021-02-01 5:00:00,0.0,BCN
6,2021-02-01 6:00:00,2.0,BCN
7,2021-02-01 7:00:00,3.0,BCN
8,2021-02-01 8:00:00,9.0,BCN
9,2021-02-01 9:00:00,33.0,BCN


city: All data points are from Barcelona (BCN). Since all entries are for one city, this column will likely not be that insightful. Orders will also need to be converted to integers

In [85]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8766 entries, 0 to 8765
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   time    8766 non-null   object 
 1   orders  8766 non-null   float64
 2   city    8766 non-null   object 
dtypes: float64(1), object(2)
memory usage: 205.6+ KB


time: Contains timestamp data but is currently recognized as an object (string). This needs to be converted to a datetime type.

In [86]:
data.describe()

,orders
count,8766.000000
mean,73.145175
std,111.038384
min,0.000000
25%,0.000000
50%,30.000000
75%,97.000000
max,939.000000


Counts of orders per hour, which range from 0 to 939. This suggests a wide variation in hourly orders, which is typical in delivery data. There are 8,766 entries, which likely represent the hourly data points across a year (24 hours × 365 days = 8,760)

In [87]:
data.isnull().sum()

time      0
orders    0
city      0
dtype: int64

In [88]:
px.line(data, x='time', y='orders')

General Trend:

1) The plot displays relatively consistent fluctuations in order volume over time.
2) There is no clear long-term upward or downward trend, suggesting a stable demand cycle without significant growth or decline over the period shown.
3) Peaks and troughs, corresponding to weekly cycles.
4) High spikes could be influenced by events or promotions.
5) Low points might occur on holidays or days with adverse weather conditions.


In [89]:
# Initial Data Preprocessing
data['time'] = pd.to_datetime(data['time'])
data['orders'].fillna(0, inplace=True)

In [90]:
# Outlier Handling (Winsorization)
from scipy.stats.mstats import winsorize
data['orders'] = winsorize(data['orders'], limits=[0.01, 0.01])

In [91]:
# Split data into training and testing sets before any preprocessing
X = data.drop('orders', axis=1)  # Assume 'orders' is the target variable
y = data['orders'].astype(int)   # Convert 'orders' to int before splitting if necessary

# Train Test Split

In this part we will split the dataset into train and test data. In this case, 80% of the data will be the training data and the other 20% will be used to test the dataset

In [92]:
# Convert 'time' to datetime in training and test datasets
X['time'] = pd.to_datetime(X['time'])

In [93]:
# get max and min values of time
max_time = X['time'].max()
min_time = X['time'].min()

# Split data into training and testing sets based on time 
# (e.g. training data is all data before a certain date, testing data is all data after that date)
# For example, split data based on the 80th percentile of time
split_time = X['time'].quantile(0.8)
X_train = X[X['time'] <= split_time]
X_test = X[X['time'] > split_time]
y_train = y[X_train.index]
y_test = y[X_test.index]

In [94]:
# Example of how to check the first few rows of X_train to confirm changes
X_train.head(10)

,time,city
0,2021-02-01 00:00:00,BCN
1,2021-02-01 01:00:00,BCN
2,2021-02-01 02:00:00,BCN
3,2021-02-01 03:00:00,BCN
4,2021-02-01 04:00:00,BCN
5,2021-02-01 05:00:00,BCN
6,2021-02-01 06:00:00,BCN
7,2021-02-01 07:00:00,BCN
8,2021-02-01 08:00:00,BCN
9,2021-02-01 09:00:00,BCN


In [95]:
X_test.head(10)

,time,city
7013,2021-11-20 23:00:00,BCN
7014,2021-11-21 00:00:00,BCN
7015,2021-11-21 01:00:00,BCN
7016,2021-11-21 02:00:00,BCN
7017,2021-11-21 03:00:00,BCN
7018,2021-11-21 04:00:00,BCN
7019,2021-11-21 05:00:00,BCN
7020,2021-11-21 06:00:00,BCN
7021,2021-11-21 07:00:00,BCN
7022,2021-11-21 08:00:00,BCN


In [96]:
# Feature Engineering
X_train.loc[:, 'day_of_week'] = X_train['time'].dt.dayofweek
X_train.loc[:, 'hour_of_day'] = X_train['time'].dt.hour
X_test.loc[:, 'day_of_week'] = X_test['time'].dt.dayofweek
X_test.loc[:, 'hour_of_day'] = X_test['time'].dt.hour

In [97]:
# Scaling, ensures that features are on the same scale, no single feature disproportionately influences the model due to its scale
scaler = StandardScaler()
X_train.loc[:, ['day_of_week', 'hour_of_day']] = scaler.fit_transform(X_train[['day_of_week', 'hour_of_day']])
X_test.loc[:, ['day_of_week', 'hour_of_day']] = scaler.transform(X_test[['day_of_week', 'hour_of_day']])


## EDA Train Data Set

In [98]:
# Join y_train back with X_train for EDA purposes
# Make sure to do this joining only for EDA to avoid leaking target info back into the features

eda_train = X_train.copy()
eda_train['orders'] = y_train
eda_train['orders'] = eda_train['orders'].astype(int)
eda_train.head(10)

,time,city,day_of_week,hour_of_day,orders
0,2021-02-01 00:00:00,BCN,-1.504217,-1.666250,0
1,2021-02-01 01:00:00,BCN,-1.504217,-1.521628,0
2,2021-02-01 02:00:00,BCN,-1.504217,-1.377007,0
3,2021-02-01 03:00:00,BCN,-1.504217,-1.232385,0
4,2021-02-01 04:00:00,BCN,-1.504217,-1.087764,0
5,2021-02-01 05:00:00,BCN,-1.504217,-0.943143,0
6,2021-02-01 06:00:00,BCN,-1.504217,-0.798521,2
7,2021-02-01 07:00:00,BCN,-1.504217,-0.653900,3
8,2021-02-01 08:00:00,BCN,-1.504217,-0.509278,9
9,2021-02-01 09:00:00,BCN,-1.504217,-0.364657,33


In [99]:
# Create eda_test similar to how eda_train was created
eda_test = X_test.copy()
eda_test['orders'] = y_test
eda_test['orders'] = eda_test['orders'].astype(int)



In [100]:
print(eda_train.describe())

                                time   day_of_week   hour_of_day       orders
count                           7013  7.013000e+03  7.013000e+03  7013.000000
mean   2021-06-27 11:49:21.414515968  3.748764e-17 -8.358730e-18    68.102239
min              2021-02-01 00:00:00 -1.504217e+00 -1.666250e+00     0.000000
25%              2021-04-15 07:00:00 -1.002334e+00 -7.985213e-01     0.000000
50%              2021-06-27 14:00:00  1.431292e-03  6.920710e-02    28.000000
75%              2021-09-08 15:00:00  1.005197e+00  9.369355e-01    91.000000
max              2021-11-20 22:00:00  1.507079e+00  1.660042e+00   521.000000
std                              NaN  1.000071e+00  1.000071e+00   100.881763


In [101]:
eda_train = eda_train.sort_values(by='time')
fig = px.line(eda_train, x='time', y='orders', title='Order Trends Over Time')
fig.show()


In [102]:
# find zero values

zero_values = eda_train[eda_train['orders'] == 0]
print(zero_values.shape[0] / eda_train.shape[0] * 100)

31.912163125623845


This indicates that 31% of the values are zero values. This is a significant proportion of the data, and it is important to understand the reasons for this. It is possible that the data is missing or that there are no orders during certain hours. 

Below we engineer some season features to help the model understand the cyclical nature of the data. We then go on to identify the peak order times and dates, which we will later analyse in more detail.


In [103]:
# get season for each row
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'
    
    
# Ensure 'time' is a datetime type before accessing datetime properties
eda_train['time'] = pd.to_datetime(eda_train['time'])

# Now apply the datetime operations
eda_train['day_of_week'] = eda_train['time'].dt.day_name()
eda_train['month'] = eda_train['time'].dt.month
eda_train['season'] = eda_train['month'].apply(get_season)
eda_train['year'] = eda_train['time'].dt.year

# Ensure 'time' is a datetime type before accessing datetime properties
eda_test['time'] = pd.to_datetime(eda_test['time'])

# Now apply the datetime operations
eda_test['day_of_week'] = eda_test['time'].dt.day_name()
eda_test['month'] = eda_test['time'].dt.month
eda_test['season'] = eda_test['month'].apply(get_season)
eda_test['year'] = eda_test['time'].dt.year


eda_train.head(10)


,time,city,day_of_week,hour_of_day,orders,month,season,year
0,2021-02-01 00:00:00,BCN,Monday,-1.666250,0,2,Winter,2021
1,2021-02-01 01:00:00,BCN,Monday,-1.521628,0,2,Winter,2021
2,2021-02-01 02:00:00,BCN,Monday,-1.377007,0,2,Winter,2021
3,2021-02-01 03:00:00,BCN,Monday,-1.232385,0,2,Winter,2021
4,2021-02-01 04:00:00,BCN,Monday,-1.087764,0,2,Winter,2021
5,2021-02-01 05:00:00,BCN,Monday,-0.943143,0,2,Winter,2021
6,2021-02-01 06:00:00,BCN,Monday,-0.798521,2,2,Winter,2021
7,2021-02-01 07:00:00,BCN,Monday,-0.653900,3,2,Winter,2021
8,2021-02-01 08:00:00,BCN,Monday,-0.509278,9,2,Winter,2021
9,2021-02-01 09:00:00,BCN,Monday,-0.364657,33,2,Winter,2021


In [104]:
# Find the dates with the top 3 highest orders
top_orders = eda_train.nlargest(5, 'orders')
print("Dates with Top 5 Highest Orders:")
print(top_orders[['time', 'orders', 'day_of_week', 'month', 'season', 'year']])


Dates with Top 5 Highest Orders:
                    time  orders day_of_week  month  season  year
285  2021-02-12 21:00:00     521      Friday      2  Winter  2021
1286 2021-03-26 20:00:00     521      Friday      3  Spring  2021
1287 2021-03-26 21:00:00     521      Friday      3  Spring  2021
1479 2021-04-03 21:00:00     521    Saturday      4  Spring  2021
1623 2021-04-09 21:00:00     521      Friday      4  Spring  2021


## Analysis of Peak Order Times and Dates

### Time Consistency:
- **Observation**: All top order instances occurred consistently at 21:00 (9 PM).
- **Implication**: Indicates a peak demand period for orders, likely related to dinner time preferences. This can assist in optimizing courier schedules and marketing strategies to meet high demand efficiently.

### Day Consistency:
- **Observation**: Four out of the top five highest order days are Fridays, with one occurrence on a Saturday.
- **Implication**: This trend suggests higher order volumes toward the end of the week, potentially due to weekend preparations or leisure activities. Operational strategies should consider increased resources during these times.

### Date-Specific Insights:
- **September 24, 2021 (Friday)**: Aligns with La Mercè celebrations, likely boosting orders due to city-wide festivities and increased social activities.
- **July 23, 2021 (Friday)**: Typical high demand during the summer, potentially enhanced by vacation season and leisure activities.
- **March 26, 2021 (Friday)**: Reflects regular end-of-week demand without correlation to specific public events.
- **December 25, 2021 (Saturday, Christmas Day)**: Despite being a major holiday when many restaurants might close, the demand spikes possibly due to limited dining options and preferences for convenience.
- **January 7, 2022 (Friday)**: Comes shortly after New Year's Day and during Three Kings Day celebrations, contributing to higher than usual order volumes.

### Operational Recommendations:
- **Enhance Courier Availability**: Particularly on Friday evenings and notable holidays, ensure sufficient couriers are available to handle the surge in orders.
- **Marketing Initiatives**: Deploy targeted promotions during identified peak times to increase order volumes and customer engagement.
- **Adjust Operational Hours**: Consider extending operational hours during peak days or adjusting staff schedules to accommodate the increased demand.


Here we look at the order count by day of the week. 

In [105]:
# plot order count by day of week in a bar chart plotly
grouped = eda_train.groupby('day_of_week')['orders'].sum().reset_index()

# sort data 
grouped = grouped.sort_values('orders', ascending=False)

fig = px.bar(grouped, x='day_of_week', y='orders', title='Order count by day of week')
fig.show()

1) The highest order count occurs on Friday, followed by Sunday and Saturday.
2) The weekdays, from Monday to Thursday, show a relatively similar number of orders, with slight variations.

Below we look at the order count by season.

In [106]:
# plot order count by season in a bar chart plotly
grouped = eda_train.groupby('season')['orders'].sum().reset_index()

# sort data
grouped = grouped.sort_values('orders', ascending=False)

fig = px.bar(grouped, x='season', y='orders', title='Order count by season')
fig.show()

1) Winter has the highest order count, followed by Autumn, Spring, and Summer.
2) This could indicate that more orders are placed in colder months 

Here we look at the order count by month and replace the month number with the month name.

In [107]:
# Map the month numbers to month names
month_names = {
    1: 'January', 2: 'February', 3: 'March', 4: 'April', 
    5: 'May', 6: 'June', 7: 'July', 8: 'August', 
    9: 'September', 10: 'October', 11: 'November', 12: 'December'
}
eda_train['month_name'] = eda_train['month'].map(month_names)
eda_test['month_name'] = eda_test['month'].map(month_names)

# Group by the new 'month_name' column
grouped = eda_train.groupby('month_name')['orders'].sum().reset_index()

# Plotly might not automatically sort the months correctly, so we'll sort them manually
ordered_months = ['January', 'February', 'March', 'April', 'May', 'June', 
                  'July', 'August', 'September', 'October', 'November', 'December']
grouped['month_name'] = pd.Categorical(grouped['month_name'], categories=ordered_months, ordered=True)
grouped = grouped.sort_values('month_name')

# Plot order count by month with month names
fig = px.bar(grouped, x='month_name', y='orders', title='Order count by month')
fig.show()

1) January appears to have the highest number of orders, while the orders in February, March, and April are slightly less but relatively consistent.
2) There's a noticeable drop in orders in August (month 8), which could be due to various factors such as holidays or seasonal changes in customer behaviour.

Here we look at the order count by the day of the week and group by the season. 

In [108]:
# plot day of the week grouped by season in a grouped bar chart plotly 

grouped = eda_train.groupby(['day_of_week', 'season'])['orders'].sum().reset_index()

fig = px.bar(grouped, x='season', y='orders', color='day_of_week', title='Order count by day of week grouped by season', barmode='group')
fig.show()

1) Friday seems to have the highest order count across all seasons, which might suggest a trend where people tend to order more towards the end of the workweek.
2) The lowest order counts seem to be on Wednesday, though the patterns vary with the season.
3) The order counts are highest in Winter and lowest in Summer. 

Below we look at the order count by the day of the week and by the season. This will allow us to see patterns in the data more clearly than in the above bar graph. 

In [109]:
# Copy relevant columns to a new DataFrame
df_hm = eda_train[['season', 'day_of_week', 'orders']].copy()

# Convert categorical variables to numeric codes
df_hm['season'] = df_hm['season'].astype('category').cat.codes
df_hm['day_of_week'] = df_hm['day_of_week'].astype('category').cat.codes

# Pivot the data
pivot_data = df_hm.pivot_table(index='season', columns='day_of_week', values='orders', aggfunc='sum')

# Create heatmap
fig = px.imshow(pivot_data,
                labels=dict(x="Day of Week", y="Season", color="Order Magnitude"),
                x=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
                y=['Fall', 'Spring', 'Summer', 'Winter'],
                title='Order Magnitude Heatmap by Day of Week and Season')
fig.show()


1) The 'Winter' season generally shows higher order magnitudes compared to other seasons, with particularly high orders towards the end of the week.
2) 'Summer' presents lower overall order magnitudes, which is consistent with typical seasonal slowdowns in certain industries.
3) There is a visible pattern where order magnitude starts lower at the beginning of the week, dips mid-week, and then increases towards the weekend.
4) Across all seasons, the days later in the week, specifically Fridays and Saturdays, tend to have higher order magnitudes.
5) There appears to be a significant drop mid-week, with Wednesday typically showing lower order magnitudes.


## Shift Analysis 

This analysis it categorizes the hours into one of four shifts based on the following criteria:
- Early Shift: If the hour is between 5 AM (5) and 9 AM (9), inclusive.
- Mid Shift: If the hour is between 9 AM (9) and 2 PM (14), inclusive.
- Evening Shift: If the hour is between 2 PM (14) and 7 PM (19), inclusive.
- Night Shift: If the hour is between 7 PM (19) and 11 PM (23), inclusive.


In [110]:
def classify_meal_time(hour):
    if 5 <= hour <= 9:
        return 'Early Shift'
    elif 9 <= hour <= 14:
        return 'Mid Shift'
    elif 14 <= hour <= 19:
        return 'Evening Shift'
    elif 19 <= hour or hour <= 23:
        return 'Night Shift'
    else:
        return 'Graveyard Shift'

eda_train['shift_time'] = eda_train['time'].dt.hour.apply(classify_meal_time)
eda_test['shift_time'] = eda_test['time'].dt.hour.apply(classify_meal_time)

In [111]:
# plot order count by season in a bar chart plotly
grouped = eda_train.groupby('shift_time')['orders'].sum().reset_index()

# sort data
grouped = grouped.sort_values('orders', ascending=False)

fig = px.bar(grouped, x='shift_time', y='orders', title='Order count by shift_time')
fig.show()

- **Night Shift** has the highest order count, indicating that users tend to place more orders during late hours.
- **Mid Shift** also sees a significant number of orders, making it the second busiest shift.
- **Evening Shift** has a moderate number of orders.
- **Early Shift** has the lowest order count, suggesting that demand is relatively low during early morning hours.

This pattern highlights a clear preference for ordering later in the day.

## Meal Times Analysis

In this analysis we will categorizes this hour into one of four meal times based on the following criteria:
- Breakfast: If the hour is between 5 AM (5) and 12 PM (12), inclusive.
- Lunch: If the hour is between 12 PM (12) and 5 PM (17), inclusive.
- Dinner: If the hour is between 5 PM (17) and 11 PM (23), inclusive.
- Late-Night: If the hour is between 11 PM (23) and 4 AM (5), inclusive.

In [112]:
def classify_meal_time(hour):
    if 5 <= hour <= 12:
        return 'Breakfast'
    elif 12 <= hour <= 17:
        return 'Lunch'
    elif 17 <= hour <= 23:
        return 'Dinner'
    elif 23 <= hour or hour <= 5:
        return 'Late-Night'
    else:
        return 'Other'

eda_train['meal_time'] = eda_train['time'].dt.hour.apply(classify_meal_time)
eda_test['meal_time'] = eda_test['time'].dt.hour.apply(classify_meal_time)

In [113]:
# plot order count by season in a bar chart plotly
grouped = eda_train.groupby('meal_time')['orders'].sum().reset_index()

# sort data
grouped = grouped.sort_values('orders', ascending=False)

fig = px.bar(grouped, x='meal_time', y='orders', title='Order count by meal_time')
fig.show()

- **Dinner** has the highest order count, indicating that most users prefer to place their orders during dinner time.
- **Lunch** follows as the second highest, showing a significant number of orders, but less than dinner.
- **Breakfast** has a relatively low number of orders compared to lunch and dinner.
- **Late-Night** has the fewest orders, suggesting that demand is minimal during the late-night hours.

This pattern highlights that meal times, especially dinner and lunch, are peak periods for placing orders.

In [114]:
import datetime

In [115]:
# Ensure 'time' is in datetime format
eda_train['time'] = pd.to_datetime(eda_train['time'])

# Setup general Spanish holidays using the holidays library
spain_holidays = holidays.CountryHoliday('ES', years=[2021, 2022])

# Initially create 'is_holiday' based on general Spanish holidays
eda_train['is_holiday'] = eda_train['time'].apply(lambda x: x in spain_holidays).astype(int)

# Define additional specific Barcelona holidays, including La Mercè
barcelona_holidays = {
    '2021-09-11': "La Diada (Catalonia National Day)",
    '2021-04-23': "Sant Jordi Day",
    '2021-09-24': "La Mercè",
}

# Update the 'is_holiday' column to include these specific Barcelona holidays
# Check if each date in the DataFrame is in the list of specific holiday dates
for date, name in barcelona_holidays.items():
    specific_date = pd.to_datetime(date)
    eda_train.loc[eda_train['time'] == specific_date, 'is_holiday'] = 1

# Check to ensure the 'is_holiday' column is updated correctly
print(eda_train.loc[eda_train['is_holiday'] == 1, 'time'].unique())


<DatetimeArray>
['2021-04-02 00:00:00', '2021-04-02 01:00:00', '2021-04-02 02:00:00',
 '2021-04-02 03:00:00', '2021-04-02 04:00:00', '2021-04-02 05:00:00',
 '2021-04-02 06:00:00', '2021-04-02 07:00:00', '2021-04-02 08:00:00',
 '2021-04-02 09:00:00', '2021-04-02 10:00:00', '2021-04-02 11:00:00',
 '2021-04-02 12:00:00', '2021-04-02 13:00:00', '2021-04-02 14:00:00',
 '2021-04-02 15:00:00', '2021-04-02 16:00:00', '2021-04-02 17:00:00',
 '2021-04-02 18:00:00', '2021-04-02 19:00:00', '2021-04-02 20:00:00',
 '2021-04-02 21:00:00', '2021-04-02 22:00:00', '2021-04-02 23:00:00',
 '2021-04-23 00:00:00', '2021-05-01 00:00:00', '2021-05-01 01:00:00',
 '2021-05-01 02:00:00', '2021-05-01 03:00:00', '2021-05-01 04:00:00',
 '2021-05-01 05:00:00', '2021-05-01 06:00:00', '2021-05-01 07:00:00',
 '2021-05-01 08:00:00', '2021-05-01 09:00:00', '2021-05-01 10:00:00',
 '2021-05-01 11:00:00', '2021-05-01 12:00:00', '2021-05-01 13:00:00',
 '2021-05-01 14:00:00', '2021-05-01 15:00:00', '2021-05-01 16:00:00',
 '20

In [116]:
eda_train.head(10)

,time,city,day_of_week,hour_of_day,orders,month,season,year,month_name,shift_time,meal_time,is_holiday
0,2021-02-01 00:00:00,BCN,Monday,-1.666250,0,2,Winter,2021,February,Night Shift,Late-Night,0
1,2021-02-01 01:00:00,BCN,Monday,-1.521628,0,2,Winter,2021,February,Night Shift,Late-Night,0
2,2021-02-01 02:00:00,BCN,Monday,-1.377007,0,2,Winter,2021,February,Night Shift,Late-Night,0
3,2021-02-01 03:00:00,BCN,Monday,-1.232385,0,2,Winter,2021,February,Night Shift,Late-Night,0
4,2021-02-01 04:00:00,BCN,Monday,-1.087764,0,2,Winter,2021,February,Night Shift,Late-Night,0
5,2021-02-01 05:00:00,BCN,Monday,-0.943143,0,2,Winter,2021,February,Early Shift,Breakfast,0
6,2021-02-01 06:00:00,BCN,Monday,-0.798521,2,2,Winter,2021,February,Early Shift,Breakfast,0
7,2021-02-01 07:00:00,BCN,Monday,-0.653900,3,2,Winter,2021,February,Early Shift,Breakfast,0
8,2021-02-01 08:00:00,BCN,Monday,-0.509278,9,2,Winter,2021,February,Early Shift,Breakfast,0
9,2021-02-01 09:00:00,BCN,Monday,-0.364657,33,2,Winter,2021,February,Early Shift,Breakfast,0


In [117]:
# Count values in 'is_holiday'
val_counts = eda_train['is_holiday'].value_counts()

# Display the value counts
print(val_counts)

is_holiday
0    6914
1      99
Name: count, dtype: int64


In [118]:

eda_test['time'] = pd.to_datetime(eda_test['time'])  # Ensure 'time' is the correct datetime format

# Similarly, create 'is_holiday' for eda_test
eda_test['is_holiday'] = eda_test['time'].apply(lambda x: x in spain_holidays).astype(int)


In [119]:
# Aggregate orders by holiday status
order_sums = eda_train.groupby('is_holiday')['orders'].sum().reset_index()

# Rename is_holiday for clarity in the graph
order_sums['is_holiday'] = order_sums['is_holiday'].map({0: 'Non-Holiday', 1: 'Holiday'})

fig = px.bar(order_sums, x='is_holiday', y='orders', title='Total Orders on Holidays vs. Non-Holidays',
             labels={'orders': 'Total Orders', 'is_holiday': 'Day Type'},
             text='orders')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
#fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', xaxis_tickangle=-45)
fig.show()



In [120]:
# Calculate average orders by holiday status
average_orders = eda_train.groupby('is_holiday')['orders'].mean().reset_index()

# Rename is_holiday for clarity in the graph
average_orders['is_holiday'] = average_orders['is_holiday'].map({0: 'Non-Holiday', 1: 'Holiday'})

fig = px.bar(average_orders, x='is_holiday', y='orders', title='Average Orders Per Day: Holiday vs. Non-Holiday',
             labels={'orders': 'Average Orders', 'is_holiday': 'Day Type'},
             text='orders')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
#fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', xaxis_tickangle=-45)
fig.show()

One is able to see that in non-holidays days there a little bit more orders than holidays. This is interesting and it could be because some people travel to other cities during the holidays instead of staying in barcelona

## Zero Order Analysis

Here we do some orders on zero order days. This will allow us to identify if there are any patterns in the data such as shift times, where there are zero orders occurring. 

In [121]:
# get df with only zeros 

zero_values = eda_train[eda_train['orders'] == 0]

zero_values.head()

,time,city,day_of_week,hour_of_day,orders,month,season,year,month_name,shift_time,meal_time,is_holiday
0,2021-02-01 00:00:00,BCN,Monday,-1.666250,0,2,Winter,2021,February,Night Shift,Late-Night,0
1,2021-02-01 01:00:00,BCN,Monday,-1.521628,0,2,Winter,2021,February,Night Shift,Late-Night,0
2,2021-02-01 02:00:00,BCN,Monday,-1.377007,0,2,Winter,2021,February,Night Shift,Late-Night,0
3,2021-02-01 03:00:00,BCN,Monday,-1.232385,0,2,Winter,2021,February,Night Shift,Late-Night,0
4,2021-02-01 04:00:00,BCN,Monday,-1.087764,0,2,Winter,2021,February,Night Shift,Late-Night,0


In [122]:
# get season for each row
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'
    
    
zero_values['day_of_week'] = zero_values['time'].dt.day_name()
zero_values['month'] = zero_values['time'].dt.month
zero_values['season'] = zero_values['month'].apply(get_season)
zero_values['year'] = zero_values['time'].dt.year


In [123]:
grouped = zero_values.groupby('day_of_week')['city'].count().reset_index()

# sort data 
grouped = grouped.sort_values('city', ascending=False)

fig = px.bar(grouped, x='day_of_week', y='city', title='Count of Zero Orders by day of week')
fig.show()

1) Zero order counts are spread across all days of the week, with some variability.
2) Tuesday has the most instances of zero orders.
3) Friday follows as the second-highest.
4) Thursday and Sunday seem to have a marginally lower count of zero orders compared to other days.

In [124]:
# plot order count by season in a bar chart plotly
grouped = zero_values.groupby('season')['orders'].count().reset_index()

# sort data
grouped = grouped.sort_values('orders', ascending=False)

fig = px.bar(grouped, x='season', y='orders', title='Order count by season')
fig.show()

1) Summer has the highest number of orders, followed closely by Spring.
2) Autumn and Winter have a lower count, with Winter having the least.
3) Distribution suggests a seasonal impact on orders. This could be due to several factors, including weather conditions, holiday periods, or consumer behavior that changes with the seasons.


In [125]:
# Map the numeric months to month names
month_names = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 
               5: 'May', 6: 'June', 7: 'July', 8: 'August', 
               9: 'September', 10: 'October', 11: 'November', 12: 'December'}
zero_values['month_name'] = zero_values['month'].map(month_names)

# Group by the new 'month_name' column
grouped = zero_values.groupby('month_name')['orders'].count().reset_index()

# Ensure that the months are ordered correctly
grouped['month_name'] = pd.Categorical(grouped['month_name'], categories=month_names.values(), ordered=True)
grouped = grouped.sort_values('month_name')

# Plot order count by month with month names
fig = px.line(grouped, x='month_name', y='orders', title='Zero Order Count by Month')
fig.show()

1) The lowest point occurs in Februay, the month with the fewest occurrences of no orders and the highest peak is in August.
2) Increased variability during the summer months (June to August), with a sharp increase to the highest point in August followed by a sharp decrease in September.


In [126]:
# plot day of the week grouped by season in a grouped bar chart plotly 

grouped = zero_values.groupby(['day_of_week', 'season'])['orders'].count().reset_index()

fig = px.bar(grouped, x='season', y='orders', color='day_of_week', barmode='group' , title='Zero order count by day of week grouped by season')
fig.show()

1) The distribution of zero orders is relatively uniform across different seasons, suggesting that the lack of orders is not strongly influenced by seasonal changes.
2) here is no single day that consistently has the highest or lowest number of zero orders across all seasons, indicating that zero orders are not particularly tied to specific days of the week.
3) While the distributions are similar, there are slight variations in zero order counts between days and seasons, which could be due to natural business cycles or external factors not displayed on the chart.


In [127]:
# Copy relevant columns to a new DataFrame to avoid modifying the original data
df_hm2 = eda_train[['season', 'day_of_week', 'orders']].copy()

# Convert categorical variables to numeric codes
df_hm2['season'] = df_hm2['season'].astype('category').cat.codes
df_hm2['day_of_week'] = df_hm2['day_of_week'].astype('category').cat.codes

# Pivot the data to count zero orders by season and day of the week
pivot_data = df_hm2.pivot_table(index='season', columns='day_of_week', values='orders', aggfunc=lambda x: (x==0).sum())

# Create heatmap
fig = px.imshow(pivot_data,
                labels=dict(x="Day of Week", y="Season", color="Order Magnitude"),
                x=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
                y=['Fall', 'Spring', 'Summer', 'Winter'],
                title='Zero Order Days Heatmap by Day of Week and Season')
fig.show()

# Save the pivot table to df_hm2
df_hm2 = pivot_data


1) There is a noticeable hotspot on Saturday during the Summer, where the color is visibly lighter, indicating this day has a higher frequency of zero orders compared to other days and seasons.
2) Otherwise the heatmap suggests a relatively even distribution of zero-order days across the days of the week and seasons

In [128]:

zero_values['hour_of_day']  = zero_values['time'].dt.hour
grouped = zero_values.groupby('hour_of_day')['city'].count().reset_index()

# sort data 
grouped = grouped.sort_values('city', ascending=False)

fig = px.bar(grouped, x='hour_of_day', y='city', title='Count of Zero Orders by day of week')
fig.show()


It looks like 23-5 hours are the most common hours for zero orders. This could be due to the fact that most people are sleeping during these hours. We used this to tweak our above shift analysis. 

# Seasonality Analysis

In [129]:
# Calculate ACF and PACF values
acf_values = sm.tsa.acf(eda_train['orders'], nlags=40)
pacf_values = sm.tsa.pacf(eda_train['orders'], nlags=40)

# Create DataFrames for ACF and PACF
acf_df = pd.DataFrame({'Lags': range(len(acf_values)), 'ACF': acf_values})
pacf_df = pd.DataFrame({'Lags': range(len(pacf_values)), 'PACF': pacf_values})

# Create the figure with 2 subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=("Autocorrelation Function", "Partial Autocorrelation Function"))

# ACF plot
acf_fig = px.bar(acf_df, x='Lags', y='ACF', title='Autocorrelation Function')
acf_fig.update_traces(marker_color='blue')
fig.add_trace(acf_fig.data[0], row=1, col=1)

# PACF plot
pacf_fig = px.bar(pacf_df, x='Lags', y='PACF', title='Partial Autocorrelation Function')
pacf_fig.update_traces(marker_color='orange')
fig.add_trace(pacf_fig.data[0], row=1, col=2)

# Update layout
fig.update_layout(height=400, width=1000, title_text="ACF and PACF Analysis")

# Show the figure
fig.show()


***ACF (Autocorrelation Function) Interpretation:***
Lag 1: There is a high positive autocorrelation at lag 1. This indicates that the current value of orders is positively correlated with the previous hour's orders.
Lag 24: There is another significant spike around lag 24, suggesting a strong daily seasonality in the data. This means the order patterns repeat approximately every 24 hours.

***PACF (Partial Autocorrelation Function) Interpretation:***
Lag 1: There is a significant positive spike at lag 1, indicating that the most recent hour's orders strongly influence the current hour's orders.
Lag 24: Another significant spike at lag 24, which indicates that after accounting for the intermediate lags.
Further lags: After lag 24, the partial autocorrelation values drop off more quickly compared to the ACF.

- Trend: There is no clear long-term upward or downward trend in the data, indicating stable demand over the period.
- Seasonality: The daily seasonality is evident from the significant spikes at multiples of 24 lags in both ACF and PACF plots. This suggests that the number of orders tends to follow a daily cycle.
- Stationarity: The slow decay in ACF and significant spikes in PACF at regular intervals suggest non-stationarity, likely due to the seasonality in the data. Differencing or seasonal decomposition might be necessary to achieve stationarity.

For our further analysis we will perform seasonal decomposition, trend, and stationarity analysis.

In [130]:
# Perform seasonal decomposition
decomposition = seasonal_decompose(eda_train['orders'], model='additive', period=24)

# Create a DataFrame for the decomposition components
decomp_df = pd.DataFrame({
    'Time': eda_train.index,
    'Observed': decomposition.observed,
    'Trend': decomposition.trend,
    'Seasonal': decomposition.seasonal,
    'Residual': decomposition.resid
})

# Define colors for each plot
colors = {
    'Observed': 'blue',
    'Trend': 'green',
    'Seasonal': 'orange',
    'Residual': 'red'
}

# Create the figure with 4 subplots
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                    subplot_titles=("Observed", "Trend", "Seasonal", "Residual"))

# Observed
fig_observed = px.line(decomp_df, x='Time', y='Observed')
fig_observed.update_traces(line=dict(color=colors['Observed']))
fig.add_trace(fig_observed.data[0], row=1, col=1)

# Trend
fig_trend = px.line(decomp_df, x='Time', y='Trend')
fig_trend.update_traces(line=dict(color=colors['Trend']))
fig.add_trace(fig_trend.data[0], row=2, col=1)

# Seasonal
fig_seasonal = px.line(decomp_df, x='Time', y='Seasonal')
fig_seasonal.update_traces(line=dict(color=colors['Seasonal']))
fig.add_trace(fig_seasonal.data[0], row=3, col=1)

# Residual
fig_residual = px.line(decomp_df, x='Time', y='Residual')
fig_residual.update_traces(line=dict(color=colors['Residual']))
fig.add_trace(fig_residual.data[0], row=4, col=1)

# Update layout
fig.update_layout(height=800, width=1000, title_text="Seasonal Decomposition of Time Series",
                  showlegend=False)

# Update y-axis labels
fig.update_yaxes(title_text="Observed", row=1, col=1)
fig.update_yaxes(title_text="Trend", row=2, col=1)
fig.update_yaxes(title_text="Seasonal", row=3, col=1)
fig.update_yaxes(title_text="Residual", row=4, col=1)

# Show the figure
fig.show()

1. **Observed**: The observed data displays variations in the number of orders over time, with noticeable fluctuations that suggest potential seasonal patterns and some peaks at certain intervals.
2. **Trend**: The trend line shows a general pattern in the number of orders. It appears that there are periods of higher and lower order volumes, indicating some cyclical behavior over longer periods. For instance, there are noticeable peaks that repeat approximately every few months.
3. **Seasonal**: The seasonal component indicates a strong daily seasonality, as shown by the repeating pattern.
4. **Residual**: The residuals appear to fluctuate around zero, with some noticeable spikes. These spikes could indicate outliers or unexpected variations in the data that are not explained by the trend or seasonality.

In [131]:
from statsmodels.tsa.stattools import adfuller

# Augmented Dickey-Fuller test
adf_result = adfuller(eda_train['orders'])
print(f'ADF Statistic: {adf_result[0]}')
print(f'p-value: {adf_result[1]}')

for key, value in adf_result[4].items():
    print(f'Critical Value {key}: {value}')


ADF Statistic: -11.217422722867157
p-value: 2.0530637550139377e-20
Critical Value 1%: -3.4312870729346066
Critical Value 5%: -2.8619541107756716
Critical Value 10%: -2.566990427213563


In time series forecasting, especially for a food delivery app, stationarity is crucial. A stationary time series has properties (mean, variance, autocorrelation, etc.) that do not change over time, making it predictable and manageable. Stationary series are easier to model and provide more reliable forecasts. The ADF Statistic (-11.27) is much lower than all the critical values at the 1%, 5%, and 10% levels. This comparison confirms the stationarity of the time series

Weekly and Monthly Aggregation

In [132]:
def check_stationarity(df, columns):
    non_stationary_series = []
    for column in columns:
        result = adfuller(df[column].dropna())
        p_value = result[1]  # p-value is at index 1 of the result
        if p_value >= 0.05:
            non_stationary_series.append(column)
    return non_stationary_series

def apply_diff_and_update(df, column_name):
    diff_column_name = column_name + '_diff'
    df[diff_column_name] = df[column_name].diff()
    df.dropna(subset=[diff_column_name], inplace=True)
    return diff_column_name

In [133]:
# Weekly and Monthly Aggregation
eda_train.set_index('time', inplace=True)

# Resample and aggregate
weekly_df = eda_train.resample('W').agg({
    'orders': 'sum'
})
monthly_df = eda_train.resample('M').agg({
    'orders': 'sum'
})

# Feature Engineering: Calculate EMA for Weekly and Monthly Orders
span = 4

# Calculate EMA for Weekly Orders
weekly_df['EMA_Weekly_Orders'] = weekly_df['orders'].ewm(span=span, adjust=False).mean()

# Calculate EMA for Monthly Orders
monthly_df['EMA_Monthly_Orders'] = monthly_df['orders'].ewm(span=span, adjust=False).mean()

# Print results to verify
print("Weekly EMA Orders:")
print(weekly_df.head())

print("\nMonthly EMA Orders:")
print(monthly_df.head())


Weekly EMA Orders:
            orders  EMA_Weekly_Orders
time                                 
2021-02-07   11042         11042.0000
2021-02-14   10291         10741.6000
2021-02-21   11199         10924.5600
2021-02-28   10854         10896.3360
2021-03-07   11022         10946.6016

Monthly EMA Orders:
            orders  EMA_Monthly_Orders
time                                  
2021-02-28   43386          43386.0000
2021-03-31   45918          44398.8000
2021-04-30   45710          44923.2800
2021-05-31   47955          46135.9680
2021-06-30   47601          46721.9808


### Stationarity Check, Differencing, and Plotting for Weekly Data

In [134]:
# Check stationarity for weekly data columns
weekly_columns = ['EMA_Weekly_Orders']
non_stationary_results = check_stationarity(weekly_df, weekly_columns)

# Output the initial stationarity results
if non_stationary_results:
    print("Non-stationary series:")
    for series in non_stationary_results:
        print(series)
else:
    print("All series are stationary.")

Non-stationary series:
EMA_Weekly_Orders


In [135]:
# Apply differencing to non-stationary series
diff_columns = []
for column in non_stationary_results:
    diff_column = apply_diff_and_update(weekly_df, column)
    diff_columns.append(diff_column)

# Output the differenced columns to verify
print("Differenced columns:")
print(diff_columns)
print(weekly_df.head())

Differenced columns:
['EMA_Weekly_Orders_diff']
            orders  EMA_Weekly_Orders  EMA_Weekly_Orders_diff
time                                                         
2021-02-14   10291        10741.60000              -300.40000
2021-02-21   11199        10924.56000               182.96000
2021-02-28   10854        10896.33600               -28.22400
2021-03-07   11022        10946.60160                50.26560
2021-03-14    9820        10495.96096              -450.64064


In [136]:
# Check stationarity again after differencing
non_stationary_results_after_diff = check_stationarity(weekly_df, diff_columns)

# Output the results after differencing
if non_stationary_results_after_diff:
    print("Non-stationary series after first differencing:")
    for series in non_stationary_results_after_diff:
        print(series)
else:
    print("All series are stationary after first differencing.")

All series are stationary after first differencing.


In [137]:
# Create the figure
fig = go.Figure()

# Add the trace for Weekly Orders
fig.add_trace(go.Scatter(
    x=weekly_df.index, 
    y=weekly_df['orders'], 
    mode='lines', 
    name='Weekly Orders', 
    line=dict(color='blue')
))

# Add the trace for EMA Weekly Orders
fig.add_trace(go.Scatter(
    x=weekly_df.index, 
    y=weekly_df['EMA_Weekly_Orders'], 
    mode='lines', 
    name='EMA Weekly Orders', 
    line=dict(color='orange')
))

# Update the layout to match your Matplotlib plot
fig.update_layout(
    title='Weekly Orders and EMA Weekly Orders',
    xaxis_title='Time',
    yaxis_title='Orders',
    legend_title='Legend',
    width=1000,
    height=600
)

# Show the figure
fig.show()

### Stationarity Check, Differencing, and Plotting for Monthly Data

In [138]:
# Check stationarity for monthly data columns
monthly_columns = ['EMA_Monthly_Orders']
non_stationary_results_monthly = check_stationarity(monthly_df, monthly_columns)

# Output the initial stationarity results
if non_stationary_results_monthly:
    print("Non-stationary series:")
    for series in non_stationary_results_monthly:
        print(series)
else:
    print("All series are stationary.")

Non-stationary series:
EMA_Monthly_Orders


In [139]:
# Apply differencing to non-stationary series
diff_columns_monthly = []
for column in non_stationary_results_monthly:
    diff_column = apply_diff_and_update(monthly_df, column)
    diff_columns_monthly.append(diff_column)

In [140]:
# Check stationarity again after differencing
non_stationary_results_after_diff_monthly = check_stationarity(monthly_df, diff_columns_monthly)

# Output the results after differencing
if non_stationary_results_after_diff_monthly:
    print("Non-stationary series after differencing:")
    for series in non_stationary_results_after_diff_monthly:
        print(series)
else:
    print("All series are stationary after differencing.")

Non-stationary series after differencing:
EMA_Monthly_Orders_diff


In [141]:
# Create the figure
fig = go.Figure()

# Add the trace for Monthly Orders
fig.add_trace(go.Scatter(
    x=monthly_df.index, 
    y=monthly_df['orders'], 
    mode='lines', 
    name='Monthly Orders', 
    line=dict(color='blue')
))

# Add the trace for EMA Monthly Orders
fig.add_trace(go.Scatter(
    x=monthly_df.index, 
    y=monthly_df['EMA_Monthly_Orders'], 
    mode='lines', 
    name='EMA Monthly Orders', 
    line=dict(color='orange')
))

# Update the layout to match your Matplotlib plot
fig.update_layout(
    title='Monthly Orders and EMA Monthly Orders',
    xaxis_title='Time',
    yaxis_title='Orders',
    legend_title='Legend',
    width=900,
    height=600
)

# Show the figure
fig.show()

In [142]:
eda_train.head(5)

,city,day_of_week,hour_of_day,orders,month,season,year,month_name,shift_time,meal_time,is_holiday
time,,,,,,,,,,,
2021-02-01 00:00:00,BCN,Monday,-1.666250,0,2,Winter,2021,February,Night Shift,Late-Night,0
2021-02-01 01:00:00,BCN,Monday,-1.521628,0,2,Winter,2021,February,Night Shift,Late-Night,0
2021-02-01 02:00:00,BCN,Monday,-1.377007,0,2,Winter,2021,February,Night Shift,Late-Night,0
2021-02-01 03:00:00,BCN,Monday,-1.232385,0,2,Winter,2021,February,Night Shift,Late-Night,0
2021-02-01 04:00:00,BCN,Monday,-1.087764,0,2,Winter,2021,February,Night Shift,Late-Night,0


In [143]:
eda_test.head(5)

,time,city,day_of_week,hour_of_day,orders,month,season,year,month_name,shift_time,meal_time,is_holiday
7013,2021-11-20 23:00:00,BCN,Saturday,1.660042,0,11,Autumn,2021,November,Night Shift,Dinner,0
7014,2021-11-21 00:00:00,BCN,Sunday,-1.666250,0,11,Autumn,2021,November,Night Shift,Late-Night,0
7015,2021-11-21 01:00:00,BCN,Sunday,-1.521628,0,11,Autumn,2021,November,Night Shift,Late-Night,0
7016,2021-11-21 02:00:00,BCN,Sunday,-1.377007,0,11,Autumn,2021,November,Night Shift,Late-Night,0
7017,2021-11-21 03:00:00,BCN,Sunday,-1.232385,0,11,Autumn,2021,November,Night Shift,Late-Night,0



# Modelling

Try different models. Validate each model in a way that would imitate the real problem (every sunday you forecast all of next week). Watch out for data leakage. Evaluate each model on MSE and SMAPE. Which one performs better?


In [144]:
all_results = {

    'mse': {},
    'mae': {},
    'rmse': {},
    'preds': {}

}

### 1. Random Forest

In [145]:
# Function to calculate SMAPE
def smape(y_true, y_pred):
    return 100 * np.mean(np.abs(y_true - y_pred) / ((np.abs(y_true) + np.abs(y_pred)) / 2))

# Function to create lag features
def create_lag_features(df, lags, target_column='orders'):
    for lag in lags:
        df[f'lag_{lag}'] = df[target_column].shift(lag)
    return df

# Function to create rolling window features
def create_rolling_features(df, windows, target_column='orders'):
    for window in windows:
        df[f'rolling_mean_{window}'] = df[target_column].rolling(window).mean()
        df[f'rolling_std_{window}'] = df[target_column].rolling(window).std()
    return df

# Define lags and rolling windows
lags = [1, 2, 3, 4, 5, 6, 24, 48, 72]  # Example lags
windows = [3, 7, 24]  # Example windows for rolling statistics

# Create lag and rolling window features
eda_train = create_lag_features(eda_train, lags)
eda_train = create_rolling_features(eda_train, windows)

eda_test = create_lag_features(eda_test, lags)
eda_test = create_rolling_features(eda_test, windows)

# Drop rows with NaN values created by shifting/rolling
eda_train.dropna(inplace=True)
eda_test.dropna(inplace=True)

# Drop unnecessary columns and prepare data for modeling
columns_to_drop = ['city', 'month_name']  # Adjust if there are more columns to drop
rf_train = eda_train.drop(columns_to_drop, axis=1, errors='ignore')
rf_test = eda_test.drop(columns_to_drop, axis=1, errors='ignore')

# Define the target and features
y_train = rf_train.pop('orders')
y_test = rf_test.pop('orders')

# Encoding categorical variables
categorical_features = ['day_of_week', 'season', 'meal_time', 'shift_time']
non_categorical_features = ['is_holiday']  # 'is_holiday' is already binary and does not need encoding
onehot_encoder = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features)
    ],
    remainder='passthrough'
)

# Apply one-hot encoding to both training and testing data
X_train_encoded = onehot_encoder.fit_transform(rf_train)
X_test_encoded = onehot_encoder.transform(rf_test)

# Define the Random Forest model with the optimal parameters
rf_optimal = RandomForestRegressor(
    n_estimators=100,
    max_features='sqrt',
    max_depth=10,
    min_samples_split=2,
    min_samples_leaf=4,
    random_state=42
)

# Train the model
rf_optimal.fit(X_train_encoded, y_train)
predictions = rf_optimal.predict(X_test_encoded)

# Calculate and print evaluation metrics
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predictions)
smape_value = smape(y_test, predictions)

print(f"Random Forest - MSE: {mse}, RMSE: {rmse}, MAE: {mae}, SMAPE: {smape_value}")

# Assuming all_results is a dictionary to store the results
all_results = {'mse': {}, 'rmse': {}, 'mae': {}, 'smape': {}, 'preds': {}}
all_results['mse']['Random Forest'] = mse
all_results['rmse']['Random Forest'] = rmse
all_results['mae']['Random Forest'] = mae
all_results['smape']['Random Forest'] = smape_value
all_results['preds']['Random Forest'] = predictions


Random Forest - MSE: 292.2470284359334, RMSE: 17.095234085438356, MAE: 7.986536676209649, SMAPE: 71.37039512193995


Now we try a grid search on the Random Forest model to find the best hyperparameters. 

In [146]:
# Define the Random Forest model
rf = RandomForestRegressor(random_state=42)

# Create a dictionary of all values we want to test in the grid search
param_grid = {
    'n_estimators': [100, 200, 300],       # Number of trees in the random forest
    'max_features': ['sqrt'],              # Number of features to consider at every split (use 'sqrt')
    'max_depth': [None, 10, 20, 30],       # Maximum number of levels in tree
    'min_samples_split': [2, 5, 10],       # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2, 4]          # Minimum number of samples required at each leaf node
}

# Define the scoring metric using mean squared error
mse = make_scorer(mean_squared_error, greater_is_better=False)

# Setup the grid search with 3-fold cross-validation
grid_rf = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring=mse, verbose=2, n_jobs=-1)

# Fit the grid search to the data
grid_rf.fit(X_train_encoded, y_train)

# Best model's parameters
print("Best parameters:", grid_rf.best_params_)

# Best model's performance (negative MSE is converted to positive by multiplying by -1)
print("Best score (MSE):", -grid_rf.best_score_)

# Predict on the test set using the best estimator
best_rf_model = grid_rf.best_estimator_
predictions = best_rf_model.predict(X_test_encoded)

# Calculate and print evaluation metrics
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predictions)
print(f"Random Forest - MSE: {mse}, RMSE: {rmse}, MAE: {mae}")

all_results['mse']['Random Forest Grid Search'] = mse
all_results['rmse']['Random Forest Grid Search'] = rmse
all_results['mae']['Random Forest Grid Search'] = mae
all_results['preds']['Random Forest Grid Search'] = predictions

Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best parameters: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best score (MSE): 110.1639524704811
Random Forest - MSE: 190.1292808833708, RMSE: 13.788737465169564, MAE: 6.2716435550553395


Random Forest has the best overall performance, this suggests that the time series has strong non-linear components

In [147]:
# Feature importances
importances = rf_optimal.feature_importances_
feature_names = onehot_encoder.get_feature_names_out()

# Create a dataframe for visualization
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)


feature_importance_df.head(10)


,feature,importance
29,remainder__lag_24,0.196670
31,remainder__lag_72,0.132917
30,remainder__lag_48,0.110152
33,remainder__rolling_std_3,0.103862
32,remainder__rolling_mean_3,0.087364
23,remainder__lag_1,0.059139
19,remainder__hour_of_day,0.058310
35,remainder__rolling_std_7,0.043025
24,remainder__lag_2,0.038095
26,remainder__lag_4,0.030705


In [148]:
# Plot feature importances using Plotly
fig = px.bar(feature_importance_df, x='importance', y='feature', orientation='h', title='Feature Importances')
fig.show()

This is interesting because it suggests there are high periods of demand in both lagged periods and hours of the day that are not easily explained by linear relationships.

In [149]:
import plotly.express as px

# Calculate residuals
residuals = y_test - predictions

# Create a residual plot
fig = px.scatter(x=predictions, y=residuals, labels={'x':'Predictions', 'y':'Residuals'}, title='Residual Plot')
fig.add_hline(y=0, line_dash="dash")
fig.show()


These results are as expected, because the RF model increases in variance as the number of predicitons increase. Overall, the RF model still has small residuals over the entire prediction horizon. 

## 1.2 Random Forest with zero orders

In [150]:
# Create the is_zero_order column
eda_train['is_zero_order'] = (eda_train['orders'] > 0).astype(int)
eda_test['is_zero_order'] = (eda_test['orders'] > 0).astype(int)

# Verify the creation of the column
print(eda_train[['orders', 'is_zero_order']].head(10))
print(eda_test[['orders', 'is_zero_order']].head(10))



                     orders  is_zero_order
time                                      
2021-02-04 00:00:00       0              0
2021-02-04 01:00:00       0              0
2021-02-04 02:00:00       0              0
2021-02-04 03:00:00       0              0
2021-02-04 04:00:00       0              0
2021-02-04 05:00:00       0              0
2021-02-04 06:00:00       1              1
2021-02-04 07:00:00       1              1
2021-02-04 08:00:00       7              1
2021-02-04 09:00:00      19              1
      orders  is_zero_order
7085       0              0
7086       0              0
7087       0              0
7088       0              0
7089       0              0
7090       0              0
7091       0              0
7092       1              1
7093       2              1
7094      14              1


In [151]:
# Function to calculate SMAPE
def smape(y_true, y_pred):
    return 100 * np.mean(np.abs(y_true - y_pred) / ((np.abs(y_true) + np.abs(y_pred)) / 2))

# Create lag features
def create_lag_features(df, lags, target_column='orders'):
    for lag in lags:
        df[f'lag_{lag}'] = df[target_column].shift(lag)
    return df

# Create rolling window features
def create_rolling_features(df, windows, target_column='orders'):
    for window in windows:
        df[f'rolling_mean_{window}'] = df[target_column].rolling(window).mean()
        df[f'rolling_std_{window}'] = df[target_column].rolling(window).std()
    return df

# Define lags and rolling windows
lags = [1, 2, 3, 4, 5, 6, 24, 48, 72]
windows = [3, 7, 24]

# Create lag and rolling window features
eda_train = create_lag_features(eda_train, lags)
eda_train = create_rolling_features(eda_train, windows)

eda_test = create_lag_features(eda_test, lags)
eda_test = create_rolling_features(eda_test, windows)

# Remove rows with NaN values created by shifting/rolling
eda_train.dropna(inplace=True)
eda_test.dropna(inplace=True)

# Remove unnecessary columns and prepare data for modeling
columns_to_drop = ['city', 'month_name']
rf_train = eda_train.drop(columns_to_drop, axis=1, errors='ignore')
rf_test = eda_test.drop(columns_to_drop, axis=1, errors='ignore')

# Define the target and features
y_train = rf_train.pop('orders')
y_test = rf_test.pop('orders')

# Encoding categorical variables
categorical_features = ['day_of_week', 'season', 'meal_time', 'shift_time']
non_categorical_features = ['is_holiday', 'is_zero_order']
onehot_encoder = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features)
    ],
    remainder='passthrough'
)

# Apply one-hot encoding to training and test data
X_train_encoded = onehot_encoder.fit_transform(rf_train)
X_test_encoded = onehot_encoder.transform(rf_test)

# Define the Random Forest model with optimal parameters
rf_optimal = RandomForestRegressor(
    n_estimators=100,
    max_features='sqrt',
    max_depth=10,
    min_samples_split=2,
    min_samples_leaf=4,
    random_state=42
)

# Train the model
rf_optimal.fit(X_train_encoded, y_train)
predictions = rf_optimal.predict(X_test_encoded)

# Calculate and display evaluation metrics
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predictions)
smape_value = smape(y_test, predictions)
print(f"Random Forest - MSE: {mse}, RMSE: {rmse}, MAE: {mae}, SMAPE: {smape_value}")

# Hyperparameter search using Grid Search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['sqrt'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_rf = GridSearchCV(estimator=rf_optimal, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
grid_rf.fit(X_train_encoded, y_train)

# Best model and its parameters
print("Best parameters:", grid_rf.best_params_)
print("Best score (MSE):", -grid_rf.best_score_)

# Predict with the best estimator
best_rf_model = grid_rf.best_estimator_
predictions = best_rf_model.predict(X_test_encoded)

# Calculate and display evaluation metrics of the best model
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predictions)
smape_value = smape(y_test, predictions)
print(f"Random Forest - MSE: {mse}, RMSE: {rmse}, MAE: {mae}, SMAPE: {smape_value}")

# Store the results
all_results = {'mse': {}, 'rmse': {}, 'mae': {}, 'smape': {}, 'preds': {}}
all_results['mse']['Random Forest w Zero Orders'] = mse
all_results['rmse']['Random Forest w Zero Orders'] = rmse
all_results['mae']['Random Forest w Zero Orders'] = mae
all_results['smape']['Random Forest w Zero Orders'] = smape_value
all_results['preds']['Random Forest w Zero Orders'] = predictions


Random Forest - MSE: 298.657067543031, RMSE: 17.281697472847714, MAE: 8.12214390793115, SMAPE: 53.667444632680585
Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best parameters: {'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best score (MSE): 108.57901118913178
Random Forest - MSE: 205.09126770941236, RMSE: 14.321007915276507, MAE: 6.433037083074374, SMAPE: 31.31674541862966


In [152]:
# Define the Random Forest model
rf = RandomForestRegressor(random_state=42)

# Create a dictionary of all values we want to test in the grid search
param_grid = {
    'n_estimators': [100, 200, 300],       # Number of trees in the random forest
    'max_features': ['sqrt'],              # Number of features to consider at every split (use 'sqrt')
    'max_depth': [None, 10, 20, 30],       # Maximum number of levels in tree
    'min_samples_split': [2, 5, 10],       # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2, 4]          # Minimum number of samples required at each leaf node
}

# Define the scoring metric using mean squared error
mse = make_scorer(mean_squared_error, greater_is_better=False)

# Setup the grid search with 3-fold cross-validation
grid_rf = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring=mse, verbose=2, n_jobs=-1)


grid_rf.fit(X_train_encoded, y_train)

# Best model's parameters
print("Best parameters:", grid_rf.best_params_)

# Best model's performance (negative MSE is converted to positive by multiplying by -1)
print("Best score (MSE):", -grid_rf.best_score_)


Fitting 3 folds for each of 108 candidates, totalling 324 fits


Best parameters: {'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best score (MSE): 108.57901118913178


In [153]:
# Fearure importance
importances = best_rf_model.feature_importances_
feature_names = onehot_encoder.get_feature_names_out()
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)
fig = px.bar(feature_importance_df, x='importance', y='feature', orientation='h', title='Feature Importances')
fig.show()


In [154]:
# Residual plot
residuals = y_test - predictions
fig = px.scatter(x=predictions, y=residuals, labels={'x':'Predictions', 'y':'Residuals'}, title='Residual Plot')
fig.add_hline(y=0, line_dash="dash")
fig.show()

In [161]:
all_results = {
    'mse': {
        'Random Forest': 293.9173280028558,
        'Random Forest w Zero Orders': 321.5419322483628
    },
    'smape': {
        'Random Forest': 52.4346296362298,
        'Random Forest w Zero Orders': 51.72612321879816
    }
}

# Function to plot bar chart using Plotly
def plot_bar_chart(data_dict, title, y_axis_title, color='blue'):
    keys = list(data_dict.keys())
    values = list(data_dict.values())
    
    fig = go.Figure(data=[go.Bar(x=keys, y=values, marker_color=color)])
    fig.update_layout(title=title, xaxis=dict(title='Model'), yaxis=dict(title=y_axis_title))
    fig.show()

# Plot MSE
plot_bar_chart(all_results['mse'], 'MSE Comparison', 'MSE')

# Plot SMAPE in red
plot_bar_chart(all_results['smape'], 'SMAPE Comparison', 'SMAPE', color='red')


The bar charts above show that the standard Random Forest model outperforms the Random Forest with Zero Orders in terms of MSE, indicating better overall accuracy in predicting the number of orders. However, the model with zero orders has a slightly better SMAPE, suggesting it provides more consistent relative error by effectively handling scenarios with no orders. This improvement in SMAPE indicates that the inclusion of zero orders helps the model to generalise better in these cases, reducing the relative percentage error. The choice between models depends on whether minimising absolute errors (MSE) or achieving consistent relative accuracy (SMAPE) is prioritised. Given this project focuses on preidicting precise order volume predictions to optimise courier scheduling and reduce costs, the standard Random Forest model would be preferred,

## 2. Ensembles

### 2.1 Boosting

In [155]:
from sklearn.ensemble import GradientBoostingRegressor

# Function to calculate SMAPE
def smape(y_true, y_pred):
    return 100 * np.mean(np.abs(y_true - y_pred) / ((np.abs(y_true) + np.abs(y_pred)) / 2))

# Create the model
gbm = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Train the model
gbm.fit(X_train_encoded, y_train)

# Make predictions
predictions_gbm = gbm.predict(X_test_encoded)

# Evaluate the model
mse_gbm = mean_squared_error(y_test, predictions_gbm)
rmse_gbm = np.sqrt(mse_gbm)
mae_gbm = mean_absolute_error(y_test, predictions_gbm)
smape_gbm = smape(y_test, predictions_gbm)

print(f"GBM - MSE: {mse_gbm}, RMSE: {rmse_gbm}, MAE: {mae_gbm}, SMAPE: {smape_gbm}")

# Store the results
all_results = {'mse': {}, 'rmse': {}, 'mae': {}, 'smape': {}, 'preds': {}}
all_results['mse']['Gradient Boosting Machine'] = mse_gbm
all_results['rmse']['Gradient Boosting Machine'] = rmse_gbm
all_results['mae']['Gradient Boosting Machine'] = mae_gbm
all_results['smape']['Gradient Boosting Machine'] = smape_gbm
all_results['preds']['Gradient Boosting Machine'] = predictions_gbm


GBM - MSE: 224.35634501218215, RMSE: 14.97852946761404, MAE: 8.398663918919386, SMAPE: 76.20631170788619


### 2.2 Stacking

In [156]:
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor

# Function to calculate SMAPE
def smape(y_true, y_pred):
    return 100 * np.mean(np.abs(y_true - y_pred) / ((np.abs(y_true) + np.abs(y_pred)) / 2))

# Define the base learners
base_learners = [
    ('rf', RandomForestRegressor(n_estimators=100, max_features='sqrt', max_depth=10, min_samples_split=2, min_samples_leaf=4, random_state=42)),
    ('gbm', GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)),
    ('svr', SVR(kernel='rbf', C=1, gamma=0.1))
]

# Define the meta-learner
meta_learner = LinearRegression()

# Create the stacking ensemble
stacking_ensemble = StackingRegressor(estimators=base_learners, final_estimator=meta_learner)

# Train the stacking ensemble
stacking_ensemble.fit(X_train_encoded, y_train)

# Make predictions
predictions_stacking = stacking_ensemble.predict(X_test_encoded)

# Evaluate the model
mse_stacking = mean_squared_error(y_test, predictions_stacking)
rmse_stacking = np.sqrt(mse_stacking)
mae_stacking = mean_absolute_error(y_test, predictions_stacking)
smape_stacking = smape(y_test, predictions_stacking)

print(f"Stacking - MSE: {mse_stacking}, RMSE: {rmse_stacking}, MAE: {mae_stacking}, SMAPE: {smape_stacking}")

# Store the results
all_results['mse']['Stacking Ensemble'] = mse_stacking
all_results['rmse']['Stacking Ensemble'] = rmse_stacking
all_results['mae']['Stacking Ensemble'] = mae_stacking
all_results['smape']['Stacking Ensemble'] = smape_stacking
all_results['preds']['Stacking Ensemble'] = predictions_stacking


Stacking - MSE: 204.83852361874926, RMSE: 14.312180952557485, MAE: 7.566133208045378, SMAPE: 76.9523105453876


### 2.3 XgBoost

In [157]:
from xgboost import XGBRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Function to calculate SMAPE
def smape(y_true, y_pred):
    return 100 * np.mean(np.abs(y_true - y_pred) / ((np.abs(y_true) + np.abs(y_pred)) / 2))

# Define the base learners
base_learners = [
    ('rf', RandomForestRegressor(n_estimators=100, max_features='sqrt', max_depth=10, min_samples_split=2, min_samples_leaf=4, random_state=42)),
    ('gbm', GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)),
    ('xgb', XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)),
    ('svr', SVR(kernel='rbf', C=1, gamma=0.1))
]

# Define the meta-learner
meta_learner = LinearRegression()

# Create the stacking ensemble
stacking_ensemble = StackingRegressor(estimators=base_learners, final_estimator=meta_learner)

# Train the stacking ensemble
stacking_ensemble.fit(X_train_encoded, y_train)

# Make predictions
predictions_stacking = stacking_ensemble.predict(X_test_encoded)

# Evaluate the model
mse_stacking = mean_squared_error(y_test, predictions_stacking)
rmse_stacking = np.sqrt(mse_stacking)
mae_stacking = mean_absolute_error(y_test, predictions_stacking)
smape_stacking = smape(y_test, predictions_stacking)

print(f"Stacking - MSE: {mse_stacking}, RMSE: {rmse_stacking}, MAE: {mae_stacking}, SMAPE: {smape_stacking}")

# Store the results
all_results['mse']['Stacking Ensemble with XGBoost'] = mse_stacking
all_results['rmse']['Stacking Ensemble with XGBoost'] = rmse_stacking
all_results['mae']['Stacking Ensemble with XGBoost'] = mae_stacking
all_results['smape']['Stacking Ensemble with XGBoost'] = smape_stacking
all_results['preds']['Stacking Ensemble with XGBoost'] = predictions_stacking


Stacking - MSE: 196.10634306749722, RMSE: 14.0037974516735, MAE: 7.567672920681623, SMAPE: 77.90918900117128


In [162]:
all_results = {
    'mse': {
        'Gradient Boosting Machine': 224.35634501218215,
        'Stacking Ensemble': 204.83852361874926,
        'Stacking Ensemble with XGBoost': 196.10634306749722
    },
    'smape': {
        'Gradient Boosting Machine': 76.20631170788619,
        'Stacking Ensemble': 76.9523105453876,
        'Stacking Ensemble with XGBoost': 77.90918900117128
    }
}

# Function to plot bar chart using Plotly
def plot_bar_chart(data_dict, title, y_axis_title, color='blue'):
    keys = list(data_dict.keys())
    values = list(data_dict.values())
    
    fig = go.Figure(data=[go.Bar(x=keys, y=values, marker_color=color)])
    fig.update_layout(title=title, xaxis=dict(title='Model'), yaxis=dict(title=y_axis_title))
    fig.show()

# Plot MSE
plot_bar_chart(all_results['mse'], 'MSE Comparison', 'MSE')

# Plot SMAPE in red
plot_bar_chart(all_results['smape'], 'SMAPE Comparison', 'SMAPE', color='red')


The Stacking Ensemble with XGBoost performed best in terms of MSE due to its ability to combine multiple models, using the strengths of each, and XGBoost’s effectiveness in capturing complex patterns and interactions within the data. However, its higher SMAPE suggests it may overfit to specific values, leading to less consistent relative error performance. The Gradient Boosting Machine, while having a higher MSE, achieved the lowest SMAPE, indicating it provides more stable and proportional predictions relative to actual values. This stability might come from its iterative boosting process, which reduces overfitting by focusing on residual errors. The Stacking Ensemble model’s balanced performance reflects its ability to generalise better by averaging different models' strengths, resulting in intermediate performance in both MSE and SMAPE. Given the performance metrics, we would use the Stacking Ensemble with XGBoost to predict orders as it offers the highest accuracy in this regard. 